# Printing

This notebook is focused on how to print numbers with formatting (width, number of digits shown, etc).

## Load Packages and Extra Functions

In [1]:
using Printf

include("jlFiles/printmat.jl")
include("jlFiles/printTeXTable.jl")

printTeXTable (generic function with 2 methods)

## Basic Printing

is done with `println()` or `display()`.

In [2]:
x = [11.11 12.12;21 22]        #a matrix to print

println(x)

[11.11 12.12; 21.0 22.0]


In [3]:
display(x)                     #gives basic formatting

2×2 Matrix{Float64}:
 11.11  12.12
 21.0   22.0

## `printmat()`

My simple function `printmat()` allows basic formatting and `printTeXTable()` prints a simple LaTeX table.

In [4]:
printmat(x,width=7,prec=2)

  11.11  12.12
  21.00  22.00



In [5]:
colNames = ["A" "B"]
rowNames = ["row 1","row 2"]
printmat(x,colNames=colNames,rowNames=rowNames,width=7,prec=2)
#printmat(x;colNames,rowNames,width=7,prec=2)    #this works too, notice the ;

           A      B
row 1  11.11  12.12
row 2  21.00  22.00



In [6]:
str = printTeXTable(x,colNames=colNames,rowNames=rowNames,width=7,prec=2);

\begin{table}
  \begin{tabular}{lrr}
     & A & B \\ \hline 
   row 1 &   11.11 &   12.12 \\ 
   row 2 &   21.00 &   22.00 \\ 
  \hline
  \end{tabular}
\end{table}


## The PrettyTables.jl Package

The [PrettyTables.jl](https://github.com/ronisbr/PrettyTables.jl) package provides powerful methods for formatted printing, including printing to LaTeX.

Try also `tf=tf_borderless`, `tf=tf_ascii_dots` or `tf=tf_unicode` (the default) as a keyword argument to modify the table border/lines.

To get a witdh of 10 and 3 digits, change to `ft_printf("%10.3f")`.

In [7]:
using PrettyTables

pretty_table(x,colNames,row_names = rowNames,tf=tf_compact,formatters=ft_printf("%5.2f"))  

 ------- ------- -------
              A       B 
 ------- ------- -------
  row 1   11.11   12.12
  row 2   21.00   22.00
 ------- ------- -------


In [8]:
pretty_table(x,colNames,formatters=ft_printf("%5.2f"),backend=:latex)   #LaTeX table

\begin{table}
  \begin{tabular}{rr}
    \hline\hline
    \textbf{A} & \textbf{B} \\\hline
    11.11 & 12.12 \\
    21.00 & 22.00 \\\hline\hline
  \end{tabular}
\end{table}


## Printing Output to a Text File

is simple. You `open()` the file, write to it and then `close()` it. The next cells demonstrate this.

In [9]:
fh = open("Results/NewTxtFile.txt", "w")        #open the file, "w" for writing

println("NewTxtFile.txt has been created in the subfolder Results. It's still empty.")

NewTxtFile.txt has been created in the subfolder Results. It's still empty.


In [10]:
println(fh,"Dogs are ")          #printing to the file, notice the fh   
println(fh,"nicer than cats.\n")

printmat(fh,x,prec=2)             #to pretty print the matrix
printmat(fh,x,colNames=colNames,rowNames=rowNames,prec=2)
printTeXTable(fh,x,colNames=colNames,rowNames=rowNames,prec=2)

pretty_table(fh,x,colNames,row_names=rowNames,formatters=ft_printf("%5.2f"))

In [11]:
close(fh)                               #close the file

println("Results/NewTxtFile.txt has been closed. Check it out. The raw contents are:\n")
println(read("Results/NewTxtFile.txt",String))

Results/NewTxtFile.txt has been closed. Check it out. The raw contents are:

Dogs are 
nicer than cats.

     11.11     12.12
     21.00     22.00

              A         B
row 1     11.11     12.12
row 2     21.00     22.00

\begin{table}
  \begin{tabular}{lrr}
     & A & B \\ \hline 
   row 1 &        11.11 &        12.12 \\ 
   row 2 &        21.00 &        22.00 \\ 
  \hline
  \end{tabular}
\end{table}
┌───────┬───────┬───────┐
│       │     A │     B │
├───────┼───────┼───────┤
│ row 1 │ 11.11 │ 12.12 │
│ row 2 │ 21.00 │ 22.00 │
└───────┴───────┴───────┘



## Redirecting the Output (extra)

In Julia 1.7+ you can also *redirect* the output from the screen to a file. This gives an simple mechanism for dumping all outputs to a file without having to insert the IO stream (called `fh` above) in all print statements.

To do that, use something like the next cell

In [12]:
redirect_stdio(stdout="Results/NewTxtFile2.txt") do     #redirect the output to a file
  println("NewTxtFile2.txt has been created in the subfolder Results. It now contains this string.")
  printmat(x,prec=2)
end  #end redirection

## Formatted Printing (extra)

using Printf in Julia 1.6+. 

(For earlier Julia versions, you may consider the [Formatting.jl](https://github.com/JuliaIO/Formatting.jl) package.)

`@printf()` prints to a file or to the screen (and IOBuffers etc), while `@sprintf()` prints to a string. 

To print with dynamic formatting (that is, where the format is decided at run-time), use `Printf.format()`. 

In [13]:
z = 3.14159      
                                         #the format string is constant
@printf("%10.2f\n",z)                    #width 10, 2 digits, floating point

str = @sprintf("%5d %g",1828,exp(1))     #width 5 and integer, compact floating point
println("my string is just this: ",str)

      3.14
my string is just this:  1828 2.71828


In [14]:
for i = 1:6
    #local width,prec,fmt,str            #only needed in script
    (width,prec) = (10,rand(0:5))       #the format string is created at run-time, here random
    fmt = Printf.Format("%$(width).$(prec)f") 
    str = Printf.format(fmt,z)
    println(str)
end    

         3
    3.1416
     3.142
       3.1
      3.14
     3.142
